[View in Colaboratory](https://colab.research.google.com/github/peaelle42/DNN_jump_start/blob/master/New_JS03_14_Simple_Text_Classification.ipynb)

# Text Classification

In [0]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Embedding, Input, GlobalAveragePooling1D
from keras.activations import relu

import numpy as np



In [2]:
print(tf.__version__)
print(tf.keras.__version__)

1.9.0-rc2
2.1.6-tf


In [0]:
# Get Data set

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [4]:
print("Training examples: {}, labels: {}".format(len(train_data), len(train_labels)))

print(train_data[0])

Training examples: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
len(train_data[0]), len(train_data[1])

(218, 189)

In [0]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [7]:
reverse_word_index[50]

'there'

In [8]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

# Sequencing and Padding

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding='post', maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post', maxlen=256)

In [10]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [11]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [0]:
Inp = Input(shape=(256,))

x = Embedding(vocab_size, 16)(Inp)
x = GlobalAveragePooling1D()(x)
x = Dense(16, activation=tf.nn.relu)(x)
output = Dense(1, activation=tf.nn.sigmoid)(x)

In [0]:
model = Model(inputs=Inp, outputs=output)

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 256, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [33]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 1s 46us/step - loss: 0.0941 - acc: 0.9755 - val_loss: 0.3089 - val_acc: 0.8837
Epoch 2/20
15000/15000 [==============================] - 1s 51us/step - loss: 0.0906 - acc: 0.9769 - val_loss: 0.3126 - val_acc: 0.8824
Epoch 3/20
15000/15000 [==============================] - 1s 54us/step - loss: 0.0873 - acc: 0.9785 - val_loss: 0.3154 - val_acc: 0.8817
Epoch 4/20
15000/15000 [==============================] - 1s 62us/step - loss: 0.0842 - acc: 0.9793 - val_loss: 0.3185 - val_acc: 0.8814
Epoch 5/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0814 - acc: 0.9805 - val_loss: 0.3220 - val_acc: 0.8818
Epoch 6/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0784 - acc: 0.9819 - val_loss: 0.3258 - val_acc: 0.8803
Epoch 7/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0761 - acc: 0.9823 - val_loss: 0.3298 - val_acc

15000/15000 [==============================] - 1s 61us/step - loss: 0.0679 - acc: 0.9854 - val_loss: 0.3411 - val_acc: 0.8793
Epoch 11/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0654 - acc: 0.9863 - val_loss: 0.3456 - val_acc: 0.8796
Epoch 12/20
15000/15000 [==============================] - 1s 63us/step - loss: 0.0630 - acc: 0.9867 - val_loss: 0.3508 - val_acc: 0.8774
Epoch 13/20
15000/15000 [==============================] - 1s 63us/step - loss: 0.0608 - acc: 0.9882 - val_loss: 0.3550 - val_acc: 0.8774
Epoch 14/20
15000/15000 [==============================] - 1s 62us/step - loss: 0.0587 - acc: 0.9881 - val_loss: 0.3592 - val_acc: 0.8772
Epoch 15/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0566 - acc: 0.9892 - val_loss: 0.3641 - val_acc: 0.8770
Epoch 16/20
15000/15000 [==============================] - 1s 61us/step - loss: 0.0551 - acc: 0.9897 - val_loss: 0.3677 - val_acc: 0.8762
Epoch 17/20
15000/15000 [=====================

15000/15000 [==============================] - 1s 62us/step - loss: 0.0493 - acc: 0.9916 - val_loss: 0.3818 - val_acc: 0.8762
Epoch 20/20
15000/15000 [==============================] - 1s 60us/step - loss: 0.0473 - acc: 0.9925 - val_loss: 0.3853 - val_acc: 0.8771


In [0]:
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 1s 34us/step
[0.30649774542808533, 0.87432]
